## Data Cleaning

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
df_trainset_values = 'data/trainset_values.csv'
df_trainset_labels = 'data/trainset_labels.csv'

df_X = pd.read_csv(df_trainset_values)
df_y = pd.read_csv(df_trainset_labels)

In [3]:
df_X.head()
df_y.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [4]:
print(df_X.shape)
print(df_y.shape)

(59400, 40)
(59400, 2)


In [5]:
drop_columns = ['payment', 'payment_type','quality_group','quantity_group','waterpoint_type_group','source_type','source_class','public_meeting', 'recorded_by','num_private','permit','scheme_management','scheme_name','subvillage']
print(len(drop_columns))
print(drop_columns)

for col in drop_columns:
    df_X.drop(columns=col,axis=1,inplace=True)

14
['payment', 'payment_type', 'quality_group', 'quantity_group', 'waterpoint_type_group', 'source_type', 'source_class', 'public_meeting', 'recorded_by', 'num_private', 'permit', 'scheme_management', 'scheme_name', 'subvillage']


In [6]:
drop_candidates = df_X.select_dtypes('object').columns
print(len(drop_candidates))
print(drop_candidates)

17
Index(['date_recorded', 'funder', 'installer', 'wpt_name', 'basin', 'region',
       'lga', 'ward', 'extraction_type', 'extraction_type_group',
       'extraction_type_class', 'management', 'management_group',
       'water_quality', 'quantity', 'source', 'waterpoint_type'],
      dtype='object')


## Imputing for Categorical

In [7]:
df_X.isnull().sum()

id                          0
amount_tsh                  0
date_recorded               0
funder                   3635
gps_height                  0
installer                3655
longitude                   0
latitude                    0
wpt_name                    0
basin                       0
region                      0
region_code                 0
district_code               0
lga                         0
ward                        0
population                  0
construction_year           0
extraction_type             0
extraction_type_group       0
extraction_type_class       0
management                  0
management_group            0
water_quality               0
quantity                    0
source                      0
waterpoint_type             0
dtype: int64

In [8]:
def impute_cat_unknown(df, col):
    '''
    Impute the value 'Unknown' for df column with na's.
    Input: df = dataframe, col = list of columns
    Output: imputed df
    '''
    for x in col:
        df[x].fillna('Unknown',inplace=True)
    return df
def impute_cat_common(df, col):
    '''
    Impute the most common value for df column with na's.
    Input: df = dataframe, col = list of columns
    Output: imputed df
    '''
    for x in col:
        df[x].apply(lambda x: x.fillna(x.value_counts().index[0]))
    return df

In [9]:
df_X = impute_cat_unknown(df_X,['installer','funder'])

In [10]:
df_X.isnull().sum()

id                       0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
basin                    0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
water_quality            0
quantity                 0
source                   0
waterpoint_type          0
dtype: int64

In [11]:
df_y['status_group'].value_counts().sort_index()

functional                 32259
functional needs repair     4317
non functional             22824
Name: status_group, dtype: int64

In [12]:
df_y['status_group'] = df_y['status_group'].apply(lambda x: x.replace(' ','_'))
df_y['status_group'].value_counts()

functional                 32259
non_functional             22824
functional_needs_repair     4317
Name: status_group, dtype: int64

In [13]:
# Dropping third label from independent variable
df_merged = df_X.merge(df_y, on='id')
df_merged = df_merged[df_merged['status_group'] != 'functional_needs_repair']

print(df_merged.shape)

df_y = df_merged['status_group']
df_X = df_merged.drop('status_group', axis=1)

(55083, 27)


In [14]:
# Converting date field to integer for use in modelling
df_X['date_recorded'] = [pd.to_datetime(x).value/100000000000 for x in df_X.date_recorded]

In [15]:
df_y

0            functional
1            functional
2            functional
3        non_functional
4            functional
              ...      
59395        functional
59396        functional
59397        functional
59398        functional
59399        functional
Name: status_group, Length: 55083, dtype: object

In [16]:
# Save for use in Analysis notebook
df_X.to_csv('data/df_X.csv')
df_y.to_csv('data/df_y.csv')